Import packages

In [1]:
import sys, os
import pathlib
# # __location__ = os.getcwd()
# # __location__ = pathlib.Path(__location__)

sys.path.append('C:\\Users\\bong\\project\\snake_RL\\dmc\\domains')
# import snake
# import snake_v2
# import snake_v3


# from snake.envs.SnakeEnv import SnakeEnv # for ray env register
# from snake_v2.envs.SnakeEnv import SnakeEnv # for ray env register
from snake_v8.envs.SnakeEnv import SnakeEnv # for ray env register
import gymnasium as gym

# import rl 알고리즘
from ray.rllib.algorithms.ppo import PPOConfig 
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig
from ray.rllib.policy.policy import Policy

from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

import numpy as np

c:\Users\Bong\anaconda3\envs\ray\lib\site-packages\ray\tune\logger\tensorboardx.py:35: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  VALID_NP_HPARAMS = (np.bool8, np.float32, np.float64, np.int32, np.int64)


Make Snake Env

In [2]:
env = gym.make('snake/SnakeEnv-v8', render_mode="human")

register_env("snake-v8", lambda config: SnakeEnv())


Load pre-learned policy

In [3]:
from ray.rllib.models import ModelCatalog
from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.models.preprocessors import get_preprocessor
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.recurrent_net import RecurrentNetwork as TorchRNN
from ray.rllib.models.tf.recurrent_net import RecurrentNetwork
from ray.rllib.utils.annotations import override
from ray.rllib.utils.framework import try_import_tf, try_import_torch

load_policy = Policy.from_checkpoint('C:\\Users\\Bong\\ray_results\\PPO_snake-v8_Base_925-1050\\checkpoint_001050')



ModuleNotFoundError: No module named 'ray.rllib.core'

In [ ]:
load_policy = Policy.from_checkpoint('C:\\Users\\Bong\\ray_results\\PPO_snake-v8_Base_925-1050\\checkpoint_001050')

In [ ]:
## For Recurrent Policy

# lstm_dim = 64
# num_lstm_cells = 2
# initial_state = np.zeros((num_lstm_cells, lstm_dim))

_state = load_policy['default_policy'].get_initial_state()
_prev_action = np.zeros(14,)
_reward = 0
_obs, _ = env.reset()
_accum_reward = 0

for epi in range(5):
    for i in range(1870):
        _act, _state, _ = load_policy['default_policy'].compute_single_action(obs=_obs.copy(), state=_state.copy(), prev_action=_prev_action.copy(), prev_reward=_reward)
        _prev_action = _act.copy()
        # print(_act)
        _obs, _reward, _done, _, _dict = env.step(_act)
        _accum_reward = _accum_reward + _reward

        if _done:
            break

        # print(str(_accum_reward)+"  \r",end='')
        # print(f"{_dict['head_rotation']}")
        # print(_dict['forward_reward'])
    _obs, _ = env.reset()
    _accum_reward = 0
    print("episode done...")
env.close()

In [ ]:
load_policy['default_policy'].model.get_parameter

Do simulation

In [ ]:
_obs, _ = env.reset()

c_reward = 0
for epi in range(10):
    for i in range(1400):
        _act = load_policy['default_policy'].compute_single_action(_obs)
        _obs, _reward, _, _, _ = env.step(_act[0])
        c_reward = c_reward + _reward
    _obs, _ = env.reset()
    print(c_reward)
    c_reward = 0
    
env.close()

In [ ]:
_state = load_policy['default_policy'].get_initial_state()
_state = np.tile(_state,(10,1))
# _state = np.concatenate(_state)

env.close()
# _obs, _ = env.reset()

_act, _state, _ = load_policy['default_policy'].compute_single_action(obs=_obs, state=_state)
